In [25]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

!pip install plotly
import plotly.graph_objects as go

In [2]:
sp100 = [
    "AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "ANET", "APA", "APD", "AVGO", "AXP", "BA", "BAC", "BIIB", "BK", "BKNG",
    "BLK", "BMY", "BSX", "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP",
    "COST", "CRM", "CSCO", "CVS", "CVX", "DHR", "DIS", "DOW", "DUK", "EMR",
    "EXC", "F", "FDX", "GD", "GE", "GILD", "GM", "GOOG", "GOOGL", "GS",
    "HON", "IBM", "INTC", "JNJ", "JPM", "KO", "LIN", "LLY", "LMT", "LOW",
    "MA", "MAR", "MCD", "MDLZ", "MDT", "MET", "MMM", "MO", "MRK", "MS",
    "MSFT", "NEE", "NFLX", "NKE", "NVDA", "ORCL", "PEP", "PFE", "PG", "PM",
    "PYPL", "QCOM", "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO",
    "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WBA",
    "WFC", "WMT", "XOM"
]

In [3]:
#takes a df and creates new columns that checks if the current 'High' price is higher than certain SMAs
def sma_filtering(df):
  filt20 = df["SMA20"] < df["High"]
  filt50 = df["SMA50"] < df["High"]
  filt100 = df["SMA100"] < df["High"]
  df["SMA20_Above"] = filt20
  df["SMA50_Above"] = filt50
  df["SMA100_Above"] = filt100
  df["above_avg"] = filt20 & filt50 & filt100
  return df

In [4]:
#takes as an input a stock name and period and returns the stock's df.
def stock_df(stock, period):
  dat = yf.Ticker(stock)
  df = dat.history(period=period).reset_index()
  df = df.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
  return df

In [5]:
# takes a given stock and iterates over 'bunch-size" stocks at a time,
# applying 'func' to each bunch independantly
def iter(stock, bunch_size, func=None):
  i=0

  #taking the stock and converting it to a df.
  if type(stock) == str:
    cur_stock_df = stock_df(stock, '1y')
  elif type(stock) == pd.DataFrame:
    cur_stock_df = stock
  else:
    return 'not a valid dataFrame'

  #iterating while the bunch size equals the 'cur_bunch'
  #if they are not equal, it means we are on the last portion and we need to stop after this one.
  while True:
    cur_bunch = cur_stock_df.iloc[i:i+bunch_size, :]
    if len(cur_bunch) < bunch_size:
      break;
    #if the user entered a function, apply it for each bunch.
    if func!=None:
      func(cur_stock_df, cur_bunch, bunch_size, i+bunch_size-1)
    i+=1
  return cur_stock_df

In [6]:
def bunch_mean(df, bunch, bunch_size, new_col_index):
  #creates a new column if not already exists.
  new_col = f"SMA{bunch_size}"
  if new_col not in df.columns:
        df[new_col] = None

  #calculates the mean of the current bunch.
  mean = bunch['High'].mean()

  #inputs the calculated mean in the corresponding column.
  df.loc[ new_col_index, f"SMA{bunch_size}"] = mean

In [7]:
top10 = []
top10_dfs = []
i=0

for stock in sp100:
  if len(top10) == 10:
    break

  df = iter(stock, 20, bunch_mean)
  df = iter(df, 50, bunch_mean)
  df = iter(df, 100, bunch_mean)
  sma_filtering(df)

  if(df.iloc[-1,-1]):
    top10.append(stock)
    top10_dfs.append(df)
  i+=1

top10

['AAPL', 'AMGN', 'AMZN', 'APA', 'AXP', 'BAC', 'BIIB', 'BK', 'BMY', 'C']

In [21]:
i=0
for df in top10_dfs:
  #
  df['Date'] = pd.to_datetime(df['Date'])
  fig = go.Figure(data = go.Scatter(x=df['Date'], y=df['High'],mode='lines', name='Price'))
  fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20')
  fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50')
  fig.add_scatter(x=df['Date'], y=df['SMA100'], mode='lines', name='SMA 100')

  #add markers
  above_sma = df['above_avg'] == True
  fig.add_scatter(
        x=df['Date'][above_sma],
        y=df['High'][above_sma],
        mode='markers',
        marker=dict(color='red', size=8, symbol='triangle-up'),
        name='Price > SMA20/50/100'
    )

  fig.update_layout(
  title=f"{top10[i]} Stock",
  xaxis_title = "Date",
  yaxis_title = "Price",
  template="plotly_dark"
  )
  #
  fig.update_xaxes(tickformat="%m-%Y")
  fig.show()
  i+=1

In [26]:
i = 0
for df in top10_dfs:
    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price'))
    fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20')
    fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50')
    fig.add_scatter(x=df['Date'], y=df['SMA100'], mode='lines', name='SMA 100')


    high_above_sma = df['High'].where(df['above_avg'], other=np.nan)

    fig.add_scatter(
        x=df['Date'],
        y=high_above_sma,
        mode='lines',
        line=dict(color='white', width=3, dash='dot'),
        name='Above all SMAs'
    )

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()
    i += 1
